In [10]:
import torch

# ok so we're implementing a custom cumsum function
# we want it to be super numerically stable

# set seed to 42
torch.manual_seed(42)

input_data = torch.randn(1000000, dtype=torch.float64, device="cuda")
input_data_fp32 = input_data.float()

# let's start by checking torch fp64 cumsum. that'll be our ground truth.
fp64_cumsum_result = torch.cumsum(input_data, dim=-1)
fp64_sum_result = torch.sum(input_data, dim=-1)

# let's see how much it diverges from doing a non-recursive for loop. my past experiments suggested this is kinda unstable.
# fp64_slow_loop_cumsum = torch.zeros_like(input_data)
# for i in range(input_data.shape[-1]):
#     fp64_slow_loop_cumsum[..., i] = torch.sum(input_data[..., :i+1], dim=-1)
# print(f"Max difference between fp64 and fp64 loop cumsum:",(fp64_cumsum - fp64_slow_loop_cumsum).max())
# then let's compare to a loop that uses an accumulator.
fp64_fast_loop_cumsum = torch.zeros_like(input_data)
fp64_fast_loop_cumsum[..., 0] = input_data[..., 0]
for i in range(1, input_data.shape[-1]):
    fp64_fast_loop_cumsum[..., i] = fp64_fast_loop_cumsum[..., i-1] + input_data[..., i]
print(f"Max difference between fp64 and fp64 fast loop cumsum:",(fp64_cumsum_result - fp64_fast_loop_cumsum).max())

# then we'll compare to torch fp32 cumsum. past experiments say this is super stable and close to fp64 torch.cumsum.
fp32_cumsum_result = torch.cumsum(input_data_fp32, dim=-1)
print(f"Max difference between fp64 and fp32 cumsum:",(fp64_cumsum_result - fp32_cumsum_result).max())

fp32_sum_result = torch.sum(input_data_fp32, dim=-1)
print(f"Max difference between fp64 and fp32 sum:",(fp64_sum_result - fp32_sum_result).max())


# then we'll test our own stuff

# first a triton kernel that uses fp64. this should be as good as torch fp64 with loop.
from stability import triton_cumsum

fp64_triton_cumsum_result = triton_cumsum(input_data)
print(f"Max difference between fp64 and fp64 triton cumsum:",(fp64_cumsum_result - fp64_triton_cumsum_result).max())

fp64_triton_cumsum_result_fp32 = fp64_triton_cumsum_result.float()
print(f"Max difference between fp64 and fp64 triton cumsum (converted to fp32):",(fp64_cumsum_result - fp64_triton_cumsum_result_fp32).max())


# then a triton kernel to do parallel scan, I think. we'll do it in fp64 at first. it should hopefully match torch fp64 cumsum super closely.

# then we'll do the same thing in fp32.

fp32_triton_cumsum_result = triton_cumsum(input_data_fp32)
print("Max diff between fp64 and fp32 triton cumsum:",(fp64_cumsum_result - fp32_triton_cumsum_result).max())

# then we'll implement our custom protect_and_attack function in fp64.

# it should be super close to torch fp64 cumsum too - about as close as our fp64 parallel scan triton kernel.

# and as we finalize each of the functions / etc, we'll move it into the stability.py file (for brevity).

Max difference between fp64 and fp64 fast loop cumsum: tensor(2.3874e-11, device='cuda:0', dtype=torch.float64)
Max difference between fp64 and fp32 cumsum: tensor(0.0007, device='cuda:0', dtype=torch.float64)
Max difference between fp64 and fp32 sum: tensor(6.2322e-05, device='cuda:0', dtype=torch.float64)
Max difference between fp64 and fp64 triton cumsum: tensor(2.3874e-11, device='cuda:0', dtype=torch.float64)
Max difference between fp64 and fp64 triton cumsum (converted to fp32): tensor(6.1035e-05, device='cuda:0', dtype=torch.float64)
Max diff between fp64 and fp32 triton cumsum: tensor(0.0216, device='cuda:0', dtype=torch.float64)


In [11]:
# ok let's make smth simple. let's just do a super simple naive algorithm.
# so we'll just try to make a rly good sum(inputs) function that takes in a list of inputs and returns a single sum.
# it will use a binary tree to accumulate them. it'll fill it out to the nearest power of 2, then treat it like some binary tree (hrrm, let's say w/ narrowing the set of active nodes by just removing the nonzero bits)
# ah what's that thing with indexing? should it be one-indexed then? b/c then you can go from the first element (i=1) to its first child (i=2) by multiplying by 2.
# yes yes that's it.
# hrrm but do we rly *want* to be doubling, or whatever? maybe that's just some random artefact of min-heaps.
# nono. I think we want zero-indexed. b/c we're gonna accumulate everything into i=0.

import torch

def bliasson_sum(x: torch.Tensor):
    closest_power_of_2 = 2**(x.shape[-1].bit_length())
    x_big = torch.zeros(closest_power_of_2, dtype=x.dtype, device=x.device)
    x_big[:x.shape[-1]] = x

    # ok so now we're going to do several rounds of summing.
    # each round will have half as many active nodes as the previous one. the final round will have 1 active node. after that round, we'll just return the sum.
    # ok so we can compute this by just using a diff set of indices each time.

    indices = torch.arange(0, closest_power_of_2)

    while len(indices) > 1:
        assert len(indices) % 2 == 0
        next_indices = indices[:len(indices)//2] * 2
        x_big[next_indices] = x_big[indices[::2]] + x_big[indices[1::2]]
        indices = next_indices
    
    return x_big[0]

assert sum(torch.tensor([1])) == 1
assert sum(torch.tensor([1, 2])) == 3
assert sum(torch.tensor([1, 2, 3])) == 6
assert sum(torch.tensor([1, 2, 3, 4])) == 10
assert sum(torch.tensor([1, 2, 3, 4, 5])) == 15
assert sum(torch.tensor([1, 2, 3, 4, 5, 6])) == 21
assert sum(torch.tensor([1, 2, 3, 4, 5, 6, 7])) == 28

def bliasson_cumsum(x: torch.Tensor):
    return torch.tensor([bliasson_sum(x[:i+1]) for i in range(x.shape[-1])])

assert (bliasson_cumsum(torch.tensor([1, 2, 3, 4, 5, 6, 7])) == torch.tensor([1, 3, 6, 10, 15, 21, 28])).all()



# now let's check the diff versus torch fp64 cumsum.

bliasson_sum_result_fp64 = bliasson_sum(input_data)
print(f"Max diff between fp64 and bliasson sum:",(fp64_sum_result - bliasson_sum_result_fp64).max())

# now let's try it in fp32.

bliasson_sum_result_fp32 = bliasson_sum(input_data_fp32)
print(f"Max diff between fp64 and bliasson sum (converted to fp32):",(fp64_sum_result - bliasson_sum_result_fp32).max())


Max diff between fp64 and bliasson sum: tensor(4.5475e-13, device='cuda:0', dtype=torch.float64)
Max diff between fp64 and bliasson sum (converted to fp32): tensor(-0.0002, device='cuda:0', dtype=torch.float64)


In [12]:
import torch
from typing import Callable

def bliasson_associative_scan(x: torch.Tensor, merge_fn: Callable=lambda l, r: l + r, dim: int=-1):
    x = x.transpose(dim, x.ndim-1) if dim != -1 else x
    *rest, n = x.shape
    bit_length = n.bit_length()
    closest_power_of_2 = 2**bit_length
    x_big = torch.zeros((*rest, closest_power_of_2), dtype=x.dtype, device=x.device)
    x_big[...,:n] = x

    # ok so now we're going to do several rounds of summing.
    # each round will have half as many active nodes as the previous one. the final round will have 1 active node. after that round, we'll just return the sum.
    # ok so we can compute this by just using a diff set of indices each time.

    indices = torch.arange(0, closest_power_of_2)

    while len(indices) > 1:
        assert len(indices) % 2 == 0
        next_indices = indices[1::2]
        x_big[...,next_indices] = merge_fn(x_big[...,indices[::2]], x_big[...,indices[1::2]])
        indices = next_indices
    
    # ok now we're going to propagate the info back down the tree, from top-down.

    for i in range(bit_length,1,-1):
        end_of_first_chunk = torch.arange(2 ** (i-1),closest_power_of_2,2 ** (i-1)) - 1
        end_of_first_half_of_second_chunk = end_of_first_chunk + 2 ** (i - 2)

        x_big[...,end_of_first_half_of_second_chunk] = merge_fn(x_big[...,end_of_first_chunk], x_big[...,end_of_first_half_of_second_chunk])
    
    raw_out = x_big[...,:n]

    return raw_out.transpose(dim, x.ndim-1) if dim != -1 else raw_out

(bliasson_associative_scan(torch.tensor([1, 2, 3, 4, 5, 6, 7])) == torch.tensor([1, 3, 6, 10, 15, 21, 28])).all()

bliasson_cumsum_result_fp64 = bliasson_associative_scan(input_data)
print(f"Max diff between fp64 and bliasson cumsum:",(fp64_cumsum_result - bliasson_cumsum_result_fp64).max())

bliasson_cumsum_result_fp32 = bliasson_associative_scan(input_data_fp32)
print(f"Max diff between fp64 and bliasson cumsum on fp32:",(fp64_cumsum_result - bliasson_cumsum_result_fp32).max())




Max diff between fp64 and bliasson cumsum: tensor(1.3642e-12, device='cuda:0', dtype=torch.float64)
Max diff between fp64 and bliasson cumsum on fp32: tensor(0.0005, device='cuda:0', dtype=torch.float64)


In [14]:
# now let's test our bliasson cumsum on our dataset of inputs that cause instability.

inputs_causing_instability = torch.load("../../inputs_causing_instability.pt")

for S_64_np, max_diff in inputs_causing_instability:
    S_64 = torch.from_numpy(S_64_np)
    FF_64 = torch.cumsum(S_64, dim=-2)
    bliasson_cumsum_result_fp64 = bliasson_associative_scan(S_64,dim=1)
    print(f"Max diff during training: {max_diff} | Max diff with bliasson cumsum: {(FF_64 - bliasson_cumsum_result_fp64).max()}")

/tmp/ipykernel_51432/580013728.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inputs_causing_instability = torch.load("../../inputs_causing_instability.pt")


Max diff during training: 1.71661376953125e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 3.4332275390625e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.52587890625e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 3.0517578125e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.239776611328125e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 7.62939453125e-06 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.33514404296875e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 5.960464477539063e-08 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.71661376953125e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.52587890625e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.5497207641601562e-05 | Max diff with bliasson cumsum: 0.0
Max diff during training: 1.049041748046875e-05 | Max diff with bliasson cumsum: 0.0
M

## Protect-and-attack algorithm

In [15]:
# ok so we have a slightly diff merge function.
# acc, for now, let's use unpack and pack.
# then we can just drop in a fully custom merge function into our bliasson algorithm.

import math
import torch
import triton
import triton.language as tl
from typing import Tuple

# Manual tuple packing by @jackd from https://github.com/openai/triton/issues/2359
@triton.jit
def unpack64(merged):
    tl.static_assert(merged.dtype == tl.uint64)
    b = (merged & 0xFFFFFFFF).to(tl.uint32).to(tl.float32, bitcast=True)
    a = (merged >> 32).to(tl.uint32).to(tl.float32, bitcast=True)
    return a, b

@triton.jit
def pack64(a, b):
    tl.static_assert(a.dtype == tl.float32)
    tl.static_assert(b.dtype == tl.float32)
    a = a.to(dtype=tl.uint32, bitcast=True).to(tl.uint64)
    a = a << 32
    b = b.to(dtype=tl.uint32, bitcast=True).to(tl.uint64)
    return a | b

# -------------------------------------------------------------------
# These are the new Triton kernels that wrap the util functions.
# They operate elementwise over flattened arrays.
# -------------------------------------------------------------------

@triton.jit
def pack_kernel(a_ptr, b_ptr, out_ptr, numel, BLOCK_SIZE: tl.constexpr):
    """
    Triton kernel for packing two FP32 arrays into one FP64 output.
    
    a_ptr : pointer to the first FP32 input tensor.
    b_ptr : pointer to the second FP32 input tensor.
    out_ptr : pointer to the output FP64 tensor. (will hold the bit-packed result)
    numel : number of elements to process
    BLOCK_SIZE : compile-time constant block size.
    """
    pid = tl.program_id(0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < numel

    # Load FP32 values from the two inputs.
    a = tl.load(a_ptr + offsets, mask=mask)
    b = tl.load(b_ptr + offsets, mask=mask)

    # Pack the two FP32 values into a single uint64.
    merged = pack64(a, b)
    # Bitcast our 64-bit unsigned integer into a FP64.
    merged_fp64 = merged.to(tl.float64, bitcast=True)
    tl.store(out_ptr + offsets, merged_fp64, mask=mask)

@triton.jit
def unpack_kernel(in_ptr, a_ptr, b_ptr, numel, BLOCK_SIZE: tl.constexpr):
    """
    Triton kernel for unpacking an FP64 array into two FP32 outputs.
    
    in_ptr : pointer to the input FP64 tensor (packed data)
    a_ptr : pointer to the first FP32 output tensor.
    b_ptr : pointer to the second FP32 output tensor.
    numel : number of elements to process
    BLOCK_SIZE : compile-time constant block size.
    """
    pid = tl.program_id(0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < numel

    # Load the packed FP64 data then bitcast it to uint64.
    merged_fp64 = tl.load(in_ptr + offsets, mask=mask)
    merged = merged_fp64.to(tl.uint64, bitcast=True)
    a, b = unpack64(merged)
    tl.store(a_ptr + offsets, a, mask=mask)
    tl.store(b_ptr + offsets, b, mask=mask)

# -------------------------------------------------------------------
# PyTorch wrappers for the new Triton kernels.
# These functions ensure the inputs are properly formatted, then launch the kernels.
# -------------------------------------------------------------------

def pack_tensors(a: torch.Tensor, b: torch.Tensor, out: torch.Tensor = None) -> torch.Tensor:
    """
    Packs two FP32 tensors elementwise into one FP64 tensor using a Triton kernel.
    
    Args:
        a (torch.Tensor): Input tensor of type torch.float32.
        b (torch.Tensor): Input tensor of type torch.float32 (must have same shape as a).
        out (torch.Tensor, optional): Output tensor to store the result.
            If None, a new tensor is allocated.
    
    Returns:
        torch.Tensor: Packed tensor of type torch.float64.
    """
    if a.shape != b.shape:
        raise ValueError("Input tensors 'a' and 'b' must have the same shape.")

    numel = a.numel()
    device = a.device

    if out is None:
        out = torch.empty_like(a, dtype=torch.float64, device=device)
    else:
        if out.shape != a.shape:
            raise ValueError("Output tensor must have the same shape as input tensors.")
        if out.dtype != torch.float64:
            raise ValueError("Output tensor must be of type torch.float64.")

    # Ensure the tensors are contiguous.
    a = a.contiguous()
    b = b.contiguous()
    out = out.contiguous()

    # Define the grid for launching the kernel.
    grid = lambda meta: (triton.cdiv(numel, meta["BLOCK_SIZE"]),)
    pack_kernel[grid](a, b, out, numel, BLOCK_SIZE=1024)
    return out

def unpack_tensor(merged: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Unpacks an FP64 (packed) tensor into two FP32 tensors using a Triton kernel.
    
    Args:
        merged (torch.Tensor): Packed input tensor of type torch.float64.
    
    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Two unpacked tensors of type torch.float32.
    """
    if merged.dtype != torch.float64:
        raise ValueError("Input tensor must be of type torch.float64.")

    numel = merged.numel()
    device = merged.device

    a = torch.empty_like(merged, dtype=torch.float32, device=device)
    b = torch.empty_like(merged, dtype=torch.float32, device=device)

    # Ensure the tensors are contiguous.
    merged = merged.contiguous()
    a = a.contiguous()
    b = b.contiguous()

    # Define the grid for launching the kernel.
    grid = lambda meta: (triton.cdiv(numel, meta["BLOCK_SIZE"]),)
    unpack_kernel[grid](merged, a, b, numel, BLOCK_SIZE=1024)
    return a, b


# tests

a,b = torch.randn(1000000,device="cuda"), torch.randn(1000000,device="cuda")

c = pack_tensors(a,b)

d,e = unpack_tensor(c)

assert (a == d).all()

In [33]:
# ok so let's define merging

def merge_fn(C1,C2):
    # unpack l into U and V
    U1,V1 = unpack_tensor(C1)
    # unpack r into W and X
    U2,V2 = unpack_tensor(C2)

    V = V1 + V2 - torch.minimum(U1,V2)
    U = U2 + torch.relu(U1-V2)

    return pack_tensors(U,V)

def protect_and_attack(A,P,dim=-1):
    A_hat = (A - P).float()
    U = torch.relu(-A_hat)
    V = torch.relu(A_hat)

    C = pack_tensors(U,V)

    C_out = bliasson_associative_scan(C,merge_fn,dim)

    U_out,V_out = unpack_tensor(C_out)

    return -1*V_out

# ok let's construct some inputs
# so for A>=0, P=0, it should match cumsum exactly.

A = torch.tensor([1,2,3],device="cuda")
P = torch.zeros_like(A,device="cuda")

torch.testing.assert_close(protect_and_attack(A,P), torch.tensor([-1.,-3.,-6.],device="cuda"))




input  tensor([0., 0., 0.], device='cuda:0') tensor([1., 2., 3.], device='cuda:0')
output tensor([0., 0., 0.], device='cuda:0') tensor([1., 3., 6.], device='cuda:0')
